<a href="https://colab.research.google.com/github/gaberman/IRPF_Calculos/blob/main/Calculos_de_Ganhos_com_A%C3%A7%C3%B5es_para_o_IRPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import sys
import pandas as pd
from google.colab import files

# Fazer upload do arquivo manualmente
uploaded = files.upload()
FILE_NAME = list(uploaded.keys())[0]  # Pega o nome do arquivo carregado

# Lendo todas as abas do arquivo Excel
try:
    xls = pd.ExcelFile(FILE_NAME)
    print("Arquivo carregado com sucesso!")
except FileNotFoundError:
    print("Erro: Certifique-se de fazer upload do arquivo Excel no Google Colab.")
    sys.exit()

# Carregar as abas relevantes
df_closed = pd.read_excel(xls, sheet_name="Closed Positions")
df_dividends = pd.read_excel(xls, sheet_name="Dividends")

# Converter datas para datetime
df_closed['Close Date'] = pd.to_datetime(df_closed['Close Date'])
df_dividends['Date of Payment'] = pd.to_datetime(df_dividends['Date of Payment'])

# Criar coluna de Ano-Mês para agrupamento
df_closed['Ano-Mês'] = df_closed['Close Date'].dt.to_period('M')
df_dividends['Ano-Mês'] = df_dividends['Date of Payment'].dt.to_period('M')

# Calcular lucro/prejuízo
if {'Open Rate', 'Close Rate', 'Units / Contracts'}.issubset(df_closed.columns):
    df_closed['Lucro/Prejuízo'] = (df_closed['Close Rate'] - df_closed['Open Rate']) * df_closed['Units / Contracts']
else:
    raise ValueError("Colunas esperadas para cálculo de lucro/prejuízo não encontradas.")

# Agrupar por Ano-Mês e Ação
df_resultado = df_closed.groupby(['Ano-Mês', 'Action'])['Lucro/Prejuízo'].sum().reset_index()

# Calcular dividendos por Ano-Mês e Ação
df_dividendos_resultado = df_dividends.groupby(['Ano-Mês', 'Instrument Name'])['Net Dividend Received (USD)'].sum().reset_index()
df_dividendos_resultado.rename(columns={'Instrument Name': 'Ação', 'Net Dividend Received (USD)': 'Dividendos'}, inplace=True)

# Juntar os resultados
df_final = pd.merge(df_resultado, df_dividendos_resultado, on=['Ano-Mês', 'Ação'], how='outer').fillna(0)

# Salvar o resultado em um novo arquivo Excel
RESULT_FILE = "resultado_imposto_renda.xlsx"
df_final.to_excel(RESULT_FILE, index=False)

print(f"Cálculo concluído! Baixe o arquivo processado: {RESULT_FILE}")


Saving etoro-account-statement-1-1-2013-3-1-2025.xlsx to etoro-account-statement-1-1-2013-3-1-2025 (8).xlsx
Arquivo carregado com sucesso!


<ipython-input-23-3a3e75259626>:22: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_closed['Close Date'] = pd.to_datetime(df_closed['Close Date'])


ValueError: time data "14/02/2017" doesn't match format "%m/%d/%Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
import pandas as pd

FILE_NAME = "etoro-account-statement-1-1-2013-3-1-2025.xlsx"

# Listar todas as páginas do arquivo Excel
xls = pd.ExcelFile(FILE_NAME)
print("Páginas encontradas:", xls.sheet_names)

# Ler todas as páginas e exibir as 5 primeiras linhas de cada uma
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    print(f"\nAba: {sheet}")
    print(df.head())
